# import

In [ ]:
# --- iPython Config --- #
from IPython import get_ipython
if 'IPython.extensions.autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')
else:
    get_ipython().run_line_magic('reload_ext', 'autoreload')
%autoreload 2

# --- System and Path --- #
import os
import sys
repo_path = os.path.dirname(os.getcwd())
if repo_path not in sys.path:
    sys.path.append(repo_path)

# --- Standard Libraries --- #
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from typing import Tuple
from sklearn.model_selection import train_test_split


# --- Custom Modules --- #
from src.config import Config
from src.data import dataloader

In [ ]:
file_path = os.path.join(repo_path, "data", "raw", "dataset.parquet")
df_dataset = dataloader.load_data(file_path)

Memory usage: Before=134.49MB -> After=65.62MB, Decreased by 51.2%
Data loaded successfully.


In [ ]:
class DataProcessor:
    def __init__(self):
        self.df_dataset = None
        self.target = ""
        self.random_state = Config.SEED # from global config

    def initial_train_test_split(self, dataset: pd.DataFrame, test_size) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Performs an initial split of the dataset into training and test sets.

        Args:
            dataset (pd.DataFrame): The dataset to be split.

        Returns:
            Tuple[pd.DataFrame, pd.DataFrame]: Training and test datasets.
        """
        X = dataset.drop(columns=[self.target])
        y = dataset[self.target]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=self.random_state, stratify=y
        )
        return X_train.join(y_train), X_test.join(y_test)


    def normalize(self, df_train, df_test) -> Tuple[pd.DataFrame, pd.DataFrame]:
        # TODO: Implement normalization
        return df_train, df_test

    def process(self, dataset: pd.DataFrame, target:str="") -> Tuple[pd.DataFrame, pd.DataFrame]:
        # init
        self.df_dataset = dataset
        self.target = target
        return df_train, df_test

data_processor = DataProcessor()
df_dataset = data_processor.process(df_dataset, target='Class')

In [15]:
df_dataset

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.099609,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.370117,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.540039,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.439941,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.969727,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568625,568625,-0.833438,0.061886,-0.899794,0.904227,-1.002401,0.481454,-0.370393,0.189694,-0.938153,...,0.167503,0.419731,1.288249,-0.900861,0.560661,-0.006018,3.308968,0.081564,4394.160156,1
568626,568626,-0.670459,-0.202896,-0.068129,-0.267328,-0.133660,0.237148,-0.016935,-0.147733,0.483894,...,0.031874,0.388161,-0.154257,-0.846452,-0.153443,1.961398,-1.528642,1.704306,4653.399902,1
568627,568627,-0.311997,-0.004095,0.137526,-0.035893,-0.042291,0.121098,-0.070958,-0.019997,-0.122048,...,0.140788,0.536523,-0.211100,-0.448909,0.540073,-0.755836,-0.487540,-0.268741,23572.849609,1
568628,568628,0.636871,-0.516970,-0.300889,-0.144480,0.131042,-0.294148,0.580568,-0.207723,0.893527,...,-0.060381,-0.195609,-0.175488,-0.554643,-0.099669,-1.434931,-0.159269,-0.076251,10160.830078,1
